In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("hw7.ipynb")

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


# CPSC 330 - Applied Machine Learning 

## Homework 7: Word embeddings and topic modeling 
**Due date: [Mar 24, 11:59 pm](https://github.com/UBC-CS/cpsc330-2024W2?tab=readme-ov-file#deliverable-due-dates-tentative).**

## Imports

In [2]:
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline

<br><br>

<!-- BEGIN QUESTION -->

<div class="alert alert-info">
    
## Submission instructions
<hr>
rubric={points}

**Please be aware that this homework assignment requires installation of several packages in your course environment. It's possible that you'll encounter installation challenges, which might be frustrating. However, remember that solving these issues is not wasting time but it is an essential skill for anyone aspiring to work in data science or machine learning.**

Follow the [homework submission instructions](https://github.com/UBC-CS/cpsc330-2024W2/blob/main/docs/homework_instructions.md). 

**You may work in a group on this homework and submit your assignment as a group.** Below are some instructions on working as a group.  
- The maximum group size is 2. 
- Use group work as an opportunity to collaborate and learn new things from each other. 
- Be respectful to each other and make sure you understand all the concepts in the assignment well. 
- It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. 
- You can find the instructions on how to do group submission on Gradescope [here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members).


When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Upload the assignment using Gradescope's drag and drop tool. Check out this [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/) if you need help with Gradescope submission.
4. Make sure that the plots and output are rendered properly in your submitted file. 
5. If the .ipynb file is too big and doesn't render on Gradescope, also upload a pdf or html in addition to the .ipynb. If the pdf or html also fail to render on Gradescope, please create two files for your homework: hw6a.ipynb with Exercise 1 and hw6b.ipynb with Exercises 2 and 3 and submit these two files in your submission.  
</div>

_Points:_ 2

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 1:  Exploring pre-trained word embeddings <a name="1"></a>
<hr>

In lecture 18, we talked about natural language processing (NLP). Using pre-trained word embeddings is very common in NLP. It has been shown that pre-trained word embeddings work well on a variety of text classification tasks. These embeddings are created by training a model like Word2Vec on a huge corpus of text such as a dump of Wikipedia or a dump of the web crawl. 

A number of pre-trained word embeddings are available out there. Some popular ones are: 

- [GloVe](https://nlp.stanford.edu/projects/glove/)
    * trained using [the GloVe algorithm](https://nlp.stanford.edu/pubs/glove.pdf) 
    * published by Stanford University 
- [fastText pre-trained embeddings for 294 languages](https://fasttext.cc/docs/en/pretrained-vectors.html) 
    * trained using the fastText algorithm
    * published by Facebook
    
In this exercise, you will be exploring GloVe Wikipedia pre-trained embeddings. The code below loads the word vectors trained on Wikipedia using an algorithm called Glove. You'll need `gensim` package in your cpsc330 conda environment to run the code below. 

```
> conda activate cpsc330
> conda install -c anaconda gensim
```

In [3]:
import gensim
import gensim.downloader

print(list(gensim.downloader.info()["models"].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [4]:
# This will take a while to run when you run it for the first time.
import gensim.downloader as api

glove_wiki_vectors = api.load("glove-wiki-gigaword-100")

In [5]:
len(glove_wiki_vectors)

400000

There are 400,000 word vectors in this pre-trained model. 

Now that we have GloVe Wiki vectors loaded in `glove_wiki_vectors`, let's explore the embeddings. 

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 Word similarity using pre-trained embeddings
rubric={points}

**Your tasks:**

- Come up with a list of 4 words of your choice and find similar words to these words using `glove_wiki_vectors` embeddings.

<div class="alert alert-warning">

Solution_1.1
    
</div>

_Points:_ 2

In [6]:
words = ["food", "pig", "man", "woman"]

In [7]:
for word in words:
    similar = glove_wiki_vectors.most_similar(word)
    result = list()
    for embedding in similar:
        result.append(embedding[0])
    
    print(word ,":" ,result)

food : ['foods', 'supplies', 'products', 'meat', 'supply', 'feed', 'medicines', 'meals', 'coffee', 'goods']
pig : ['pigs', 'cow', 'sheep', 'hog', 'chickens', 'meat', 'cows', 'chicken', 'goat', 'dog']
man : ['woman', 'boy', 'one', 'person', 'another', 'old', 'life', 'father', 'turned', 'who']
woman : ['girl', 'man', 'mother', 'boy', 'she', 'child', 'wife', 'her', 'herself', 'daughter']


In [8]:
...

Ellipsis

In [9]:
...

Ellipsis

In [10]:
...

Ellipsis

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.2 Word similarity using pre-trained embeddings
rubric={points}

**Your tasks:**

1. Calculate cosine similarity for the following word pairs (`word_pairs`) using the [`similarity`](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) method of `glove_wiki_vectors`.

In [11]:
word_pairs = [
    ("coast", "shore"),
    ("clothes", "closet"),
    ("old", "new"),
    ("smart", "intelligent"),
    ("dog", "cat"),
    ("tree", "lawyer"),
]

<div class="alert alert-warning">

Solution_1.2
    
</div>

_Points:_ 2

In [12]:
for pair in word_pairs:
    result = glove_wiki_vectors.similarity(pair[0], pair[1])
    print(pair, result)

('coast', 'shore') 0.70002717
('clothes', 'closet') 0.546276
('old', 'new') 0.6432488
('smart', 'intelligent') 0.7552732
('dog', 'cat') 0.8798075
('tree', 'lawyer') 0.076719455


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Representation of all words in English
rubric={points}

**Your tasks:**

1. The vocabulary size of Wikipedia embeddings is quite large. The `test_words` list below contains a few new words (called neologisms) and biomedical domain-specific abbreviations. Write code to check whether `glove_wiki_vectors` have representation for these words or not. 
> If a given word `word` is in the vocabulary, `word in glove_wiki_vectors` will return True. 

In [13]:
test_words = [
    "covididiot",
    "fomo",
    "frenemies",
    "anthropause",
    "photobomb",
    "selfie",
    "pxg",  # Abbreviation for pseudoexfoliative glaucoma
    "pacg",  # Abbreviation for primary angle closure glaucoma
    "cct",  # Abbreviation for central corneal thickness
    "escc",  # Abbreviation for esophageal squamous cell carcinoma
]

<div class="alert alert-warning">

Solution_1_3
    
</div>

_Points:_ 2

In [14]:
for word in test_words:
    response = word in glove_wiki_vectors
    print(word, response)

covididiot False
fomo False
frenemies True
anthropause False
photobomb False
selfie False
pxg False
pacg False
cct True
escc True


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.4 Stereotypes and biases in embeddings
rubric={points}

Word vectors contain lots of useful information. But they also contain stereotypes and biases of the texts they were trained on. In the lecture, we saw an example of gender bias in Google News word embeddings. Here we are using pre-trained embeddings trained on Wikipedia data. 

**Your tasks:**

1. Explore whether there are any worrisome biases or stereotypes present in these embeddings by trying out at least 4 examples. You can use the following two methods or other methods of your choice to explore this. 
    - the `analogy` function below which gives word analogies (an example shown below)
    - [similarity](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) or [distance](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=distance#gensim.models.keyedvectors.KeyedVectors.distances) methods (an example is shown below)

> Note that most of the recent embeddings are de-biased. But you might still observe some biases in them. Also, not all stereotypes present in pre-trained embeddings are necessarily bad. But you should be aware of them when you use them in your models. 

In [15]:
def analogy(word1, word2, word3, model=glove_wiki_vectors):
    """
    Returns analogy word using the given model.

    Parameters
    --------------
    word1 : (str)
        word1 in the analogy relation
    word2 : (str)
        word2 in the analogy relation
    word3 : (str)
        word3 in the analogy relation
    model :
        word embedding model

    Returns
    ---------------
        pd.dataframe
    """
    print("%s : %s :: %s : ?" % (word1, word2, word3))
    sim_words = model.most_similar(positive=[word3, word2], negative=[word1])
    return pd.DataFrame(sim_words, columns=["Analogy word", "Score"])

Examples of using analogy to explore biases and stereotypes.  

In [16]:
analogy("man", "doctor", "woman")

man : doctor :: woman : ?


,Analogy word,Score
0,nurse,0.773523
1,physician,0.718943
2,doctors,0.682433
3,patient,0.675068
4,dentist,0.672603
5,pregnant,0.664246
6,medical,0.652045
7,nursing,0.645348
8,mother,0.639333
9,hospital,0.638750


In [17]:
glove_wiki_vectors.similarity("aboriginal", "success")

0.1428324

In [18]:
glove_wiki_vectors.similarity("white", "success")

0.351824

<div class="alert alert-warning">

Solution_1_4
    
</div>

_Points:_ 4

In [19]:
analogy("father", "schoolteacher", "mother")

father : schoolteacher :: mother : ?


,Analogy word,Score
0,housewife,0.791876
1,homemaker,0.787397
2,waitress,0.640994
3,seamstress,0.635987
4,businesswoman,0.622617
5,housekeeper,0.620299
6,nurse,0.619902
7,widowed,0.616345
8,beautician,0.606480
9,pensioner,0.601561


In [20]:
analogy("black", "bad", "white")

black : bad :: white : ?


,Analogy word,Score
0,worse,0.743016
1,good,0.722864
2,things,0.714939
3,anything,0.700573
4,nothing,0.691554
5,going,0.689290
6,sure,0.684152
7,thing,0.681583
8,n't,0.677742
9,wrong,0.675569


In [21]:
glove_wiki_vectors.similarity("asian", "smart")


0.19219656

In [22]:
glove_wiki_vectors.similarity("white", "smart")


0.3288475

In [23]:
analogy("russian", "aggressive", "american")


russian : aggressive :: american : ?


,Analogy word,Score
0,kind,0.558982
1,strong,0.550317
2,approach,0.547907
3,aggressively,0.532748
4,pace,0.527673
5,action,0.526564
6,way,0.525899
7,successful,0.522021
8,more,0.521729
9,better,0.517930


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.5 Discussion
rubric={points}

**Your tasks:**
1. Discuss your observations from 1.4. Are there any worrisome biases in these embeddings trained on Wikipedia?   
2. Give an example of how using embeddings with biases could cause harm in the real world.

<div class="alert alert-warning">

Solution_1_5
    
</div>

_Points:_ 4

There are some worrisome ethnic biases and gender stereotypes in professions based on the results. The results show a higher association of "smart" with "white" compared to "asian" which contradicts the common stereotypes, however, the mismatch still highlights problematic biases. The association of "father" to "schoolteacher" while "mother" to "housewife"  reinforces gender stereotypes, in family roles. Also, terms like "homemaker" and "housewife" are associated to "mother" while the father is working. 

In the real world, Such embeddings could influence hiring algorithms and fewer women end up being in leadership or higher power positions. Similarly, candidates from certain ethnic or religious backgrounds might be unfairly filtered out or deprioritized if the embeddings associate their identity with negative traits like “lazy,” “dangerous,” or “unqualified.”



<!-- END QUESTION -->

<br><br><br><br>

## Exercise 2: Topic modeling 

The goal of topic modeling is discovering high-level themes in a large collection of texts. 

In this homework, you will explore topics in [the 20 newsgroups text dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html) using `scikit-learn`'s `LatentDirichletAllocation` (LDA) model. 

Usually, topic modeling is used for discovering abstract "topics" that occur in a collection of documents when you do not know the actual topics present in the documents. But 20 newsgroups text dataset is labeled with categories (e.g., sports, hardware, religion), and you will be able to cross-check the topics discovered by your model with these available topics. 

The starter code below loads the train and test portion of the data and convert the train portion into a pandas DataFrame. For speed, we will only consider documents with the following 8 categories. 

In [24]:
from sklearn.datasets import fetch_20newsgroups

In [25]:
cats = [
    "rec.sport.hockey",
    "rec.sport.baseball",
    "soc.religion.christian",
    "alt.atheism",
    "comp.graphics",
    "comp.windows.x",
    "talk.politics.mideast",
    "talk.politics.guns",
]  # We'll only consider these categories out of 20 categories for speed.

newsgroups_train = fetch_20newsgroups(
    subset="train", remove=("headers", "footers", "quotes"), categories=cats
)
X_news_train, y_news_train = newsgroups_train.data, newsgroups_train.target
df = pd.DataFrame(X_news_train, columns=["text"])
df["target"] = y_news_train
df["target_name"] = [
    newsgroups_train.target_names[target] for target in newsgroups_train.target
]
df

,text,target,target_name
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics
3,"The following problem is really bugging me,\na...",2,comp.windows.x
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast
...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x


In [26]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.windows.x',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast']

<br><br>

<!-- BEGIN QUESTION -->

### 2.1 Preprocessing using [spaCy](https://spacy.io/)
rubric={points}

Preprocessing is a crucial step before carrying out topic modeling and it markedly affects topic modeling results. In this exercise, you'll prepare the data using [spaCy](https://spacy.io/) for topic modeling. 

**Your tasks:** 

- Write code using [spaCy](https://spacy.io/) to preprocess the `text` column in the given dataframe `df` and save the processed text in a new column called `text_pp` within the same dataframe.

If you do not have spaCy in your course environment, you'll have to [install it](https://spacy.io/usage) and download the pretrained model en_core_web_md. 

`python -m spacy download en_core_web_md`


Note that there is no such thing as "perfect" preprocessing. You'll have to make your own judgments and decisions on which tokens are likely to be more informative for the given task. Some common text preprocessing steps for topic modeling include: 
- getting rid of slashes, new-line characters, or any other non-informative characters
- sentence segmentation and tokenization      
- replacing urls, email addresses, or numbers with generic tokens such as "URL",  "EMAIL", "NUM". 
- getting rid of other fairly unique tokens which are not going to help us in topic modeling  
- excluding stopwords and punctuation 
- lemmatization


> Check out [these available attributes](https://spacy.io/api/token#attributes) for `token` in spaCy which might help you with preprocessing. 

> You can also get rid of words with specific POS tags. [Here](https://universaldependencies.org/u/pos/) is the list of part-of-speech tags used in spaCy. 

> You may have to use regex to clean text before passing it to spaCy. Also, you might have to go back and forth between preprocessing in this exercise and and topic modeling in Exercise 2 before finalizing preprocessing steps. 

> Note that preprocessing the corpus might take some time. So here are a couple of suggestions: 1) During the debugging phase, work on a smaller subset of the data. 2) Once you finalize the preprocessing part, you might want to save the preprocessed data in a CSV and work with this CSV so that you don't run the preprocessing part every time you run the notebook. 
 


In [27]:
import spacy
nlp = spacy.load("en_core_web_md", disable=["parser", "ner"])

<div class="alert alert-warning">

Solution_2_1
    
</div>

_Points:_ 8

In [28]:
import re
from tqdm import tqdm

def preprocess_text(text):
    # Replace URLs, emails, and numbers
    text = re.sub(r"http\S+", "URL", text)  # Replace URLs with "URL"
    text = re.sub(r"\S+@\S+", "EMAIL", text)  # Replace email addresses with "EMAIL"
    text = re.sub(r"\d+", "NUM", text) 
    
    # Process the text using the spaCy model
    doc = nlp(text)
    
    # Filter tokens
    tokens = [
        token.lemma_.lower()  # Use the lemmatized form in lowercase
        for token in doc
        if not token.is_stop  # Exclude stopwords
        and not token.is_punct  # Exclude punctuation
        and not token.is_space  # Exclude spaces
        and token.is_alpha  # Include only alphabetic tokens
        and token.pos_ not in {"PROPN"}  # Exclude proper nouns
    ]
    
    return " ".join(tokens)

In [29]:
# Apply preprocessing to the text column
tqdm.pandas(desc="Preprocessing")  # Progress bar for tracking
df["text_pp"] = df["text"].progress_apply(preprocess_text)

Preprocessing: 100%|██████████████████████████████████████████████| 4563/4563 [01:29<00:00, 51.17it/s]


In [30]:
...

Ellipsis

In [31]:
df.iloc[2:6]

,text,target,target_name,text_pp
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics,hand support idea have newsgroup aspect graphi...
3,"The following problem is really bugging me,\na...",2,comp.windows.x,follow problem bug appreciate help create wind...
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast,late spokesman tell journalist close air space...
5,"Hi,\n I'd like to subscribe to Leadership Ma...",5,soc.religion.christian,hi like subscribe wonder disk instead paper ha...


In [32]:
df.to_csv("preprocessed_newsgroups.csv", index=False)

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Justification
rubric={points}

**Your tasks:**

- Outline the preprocessing steps you carried out in the previous exercise (bullet point format is fine), providing a brief justification when necessary. 

> You might want to wait to answer this question till you are done with Exercise 2 and you have finalized the preprocessing steps in 2.1. 

<div class="alert alert-warning">

Solution_2_2
    
</div>

_Points:_ 2

### Removing URLs:

Replaced all URLs with the token "URL"

Justification: URLs are not useful for most NLP tasks and can add noise. Replacing them with a placeholder preserves the information that a link was present without keeping its content.

### Replacing email addresses:

Replaced all email addresses with the token "EMAIL"

Justification: Like URLs, emails are often irrelevant for text understanding and can be sensitive. Masking them helps anonymize the text.

### Replacing numbers:

Replaced all numeric sequences with the token "NUM"

Justification: Raw numbers may not be meaningful out of context and replacing them helps generalize patterns without losing that a quantity existed.

### Tokenization and Lemmatization:

Broke text into tokens and reduced each to its lemma in lowercase

### Removing stopwords:

Excluded common stopwords like "the" and "is"

Justification: These words usually don't add significant semantic meaning.

### Removing punctuation and spaces:

Excluded all punctuation and whitespace tokens

Justification: These elements are generally not helpful for tasks like classification or topic modeling.

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.3 Build a topic model using sklearn's LatentDirichletAllocation
rubric={points}

**Your tasks:**

1. Build LDA models on the preprocessed data using using [sklearn's `LatentDirichletAllocation`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html) and random state 42. Experiment with a few values for the number of topics (`n_components`). Pick a reasonable number for the number of topics and briefly justify your choice.

<div class="alert alert-warning">

Solution_2_3
    
</div>

_Points:_ 4

In [33]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pandas as pd

vectorizer = CountVectorizer(max_df=0.95, min_df=2)  # remove rare and overly common words
doc_term_matrix = vectorizer.fit_transform(df["text_pp"])

lda_models = {}
for n_topics in [5, 10, 15]:
    lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
    lda.fit(doc_term_matrix)
    lda_models[n_topics] = lda
def display_topics(model, feature_names, num_words=10):
    for topic_idx, topic in enumerate(model.components_):
        top_words = [feature_names[i] for i in topic.argsort()[:-num_words - 1:-1]]
        print(f"Topic {topic_idx + 1}: {', '.join(top_words)}\n")

feature_names = vectorizer.get_feature_names_out()

for n_topics, model in lda_models.items():
    print(f"\n--- Top words for {n_topics} Topics ---")
    display_topics(model, feature_names, num_words=10)



--- Top words for 5 Topics ---
Topic 1: game, team, year, play, good, player, win, think, season, get

Topic 2: gun, know, people, go, think, say, like, come, weapon, right

Topic 3: think, people, believe, know, question, thing, say, time, good, way

Topic 4: people, armenian, say, turkish, right, kill, israeli, government, state, come

Topic 5: file, program, use, image, window, available, include, run, work, server


--- Top words for 10 Topics ---
Topic 1: game, team, year, player, good, play, win, season, think, get

Topic 2: period, play, goal, shot, pp, get, second, fire, puck, penalty

Topic 3: believe, think, know, exist, people, faith, religion, belief, question, atheist

Topic 4: armenian, people, turkish, village, genocide, kill, population, attack, massacre, government

Topic 5: turkish, armenian, greek, jewish, people, book, point, num, government, german

Topic 6: say, know, go, people, come, think, tell, time, like, want

Topic 7: think, people, right, gun, law, like, 

I think I will go forward with value 10 because at 5 broad and vague wheres at 15, topics get redundant.

In [34]:
# final LDA with n_components = 10
lda = LatentDirichletAllocation(n_components=10, random_state=42)
lda.fit(doc_term_matrix)



LatentDirichletAllocation(random_state=42)

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.4 Exploring word topic association
rubric={points}

**Your tasks:**
1. For the number of topics you picked in the previous exercise, show top 10 words for each of your topics and suggest labels for each of the topics (similar to how we came up with labels "health and nutrition", "fashion", and "machine learning" in the toy example we saw in class). 

> If your topics do not make much sense, you might have to go back to preprocessing in Exercise 2.1, improve it, and train your LDA model again. 

<div class="alert alert-warning">

Solution_2_4
    
</div>

_Points:_ 5

In [35]:
def display_topics(model, feature_names, num_words=10):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic {topic_idx}:")
        print(", ".join([feature_names[i] for i in topic.argsort()[-num_words:]]))
        print("\n")

feature_names = vectorizer.get_feature_names_out()

display_topics(lda, feature_names)

Topic 0:
get, think, season, win, play, good, player, year, team, game


Topic 1:
penalty, puck, fire, second, get, pp, shot, goal, play, period


Topic 2:
atheist, question, belief, religion, faith, people, exist, know, think, believe


Topic 3:
government, massacre, attack, population, kill, genocide, village, turkish, people, armenian


Topic 4:
german, government, num, point, book, people, jewish, greek, armenian, turkish


Topic 5:
want, like, time, tell, think, come, people, go, know, say


Topic 6:
thing, know, mean, fact, like, law, gun, right, people, think


Topic 7:
case, kill, right, crime, israeli, state, law, firearm, weapon, gun


Topic 8:
entry, server, run, include, available, window, image, use, program, file


Topic 9:
find, come, jewish, time, book, think, right, argument, know, people




Topic 0: General Sports Commentary

Topic 1: Ice Hockey/Game Analysis

Topic 2: Religion and Belief Systems

Topic 3: Conflict and Genocide (Armenian Genocide)

Topic 4: Historical and Ethnic Politics

Topic 5: Everyday Conversation

Topic 6: Gun Rights and Legal Debates

Topic 7: Crime, Violence, and Law Enforcement

Topic 8: Software and Programming

Topic 9: Philosophy and Ethics

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.5 Exploring document topic association
rubric={points}

**Your tasks:**
1. Show the document topic assignment of the first five documents from `df`. 
2. Comment on the document topic assignment of the model. 

<div class="alert alert-warning">

Solution_2_5
    
</div>

_Points:_ 5

In [36]:
#1.
doc_topic_dists = lda_models[10].transform(doc_term_matrix)

# Show dominant topic for the first 5 documents
for i in range(5):
    topic_probs = doc_topic_dists[i]
    dominant_topic = topic_probs.argmax()
    print(f"Document {i}: Dominant Topic = {dominant_topic}, Distribution = {topic_probs.round(3)}")


Document 0: Dominant Topic = 7, Distribution = [0.002 0.04  0.079 0.002 0.002 0.172 0.284 0.415 0.002 0.002]
Document 1: Dominant Topic = 4, Distribution = [0.001 0.001 0.001 0.001 0.784 0.067 0.001 0.001 0.139 0.001]
Document 2: Dominant Topic = 2, Distribution = [0.002 0.002 0.44  0.002 0.002 0.103 0.188 0.002 0.258 0.002]
Document 3: Dominant Topic = 8, Distribution = [0.004 0.004 0.004 0.004 0.004 0.098 0.004 0.004 0.87  0.004]
Document 4: Dominant Topic = 3, Distribution = [0.003 0.003 0.003 0.578 0.003 0.003 0.003 0.399 0.003 0.003]


2. In my opinion, the model performs well overall, especially with politically charged or technical content. Document 0, 3, and 4 are well-classified, showing that LDA can effectively capture clear themes. However, documents with more general language or mixed themes (like Documents 1 and 2) may confuse the model, leading to mismatches. 

<!-- END QUESTION -->

<br><br><br><br>

<!-- BEGIN QUESTION -->

## Exercise 3: Short answer questions 
<hr>

rubric={points}

1. Briefly explain how content-based filtering works in the context of recommender systems. 
2. Discuss at least two negative consequences of recommender systems.
3. What is transfer learning in natural language processing? Briefly explain.     

<div class="alert alert-warning">

Solution_3
    
</div>

_Points:_ 6

1. Content-based filtering recommends items to a user by analyzing the features of items they have previously liked and matching those features with other items. For instanc, if a user watches rom-com movies with certain actors or directors, the system will recommend other movies with similar characteristics. It relies on attributes like genre, keywords and user preferences, rather than data from other users.
2. Filter bubbles: Recommender systems can repeatedly show users content similar to what they already like, limiting exposure to diverse viewpoints or new topics. This reinforces existing preferences.
Overfitting to short-term behavior: Some systems may overemphasize recent clicks or views, leading to recommendations that don’t truly reflect the user’s broader or long-term interests.
4. Transfer learning in natural language processing involves taking a pre-trained model (usually trained on a large corpus like Wikipedias) and fine-tuning it on a specific/smaller task such as sentiment analysis or question answering.

<!-- END QUESTION -->

<br><br><br><br>

**Before submitting your assignment, please make sure you have followed all the instructions in the Submission instructions section at the top.** 

![](img/eva-well-done.png)